# Imports

In [1]:
# Import required libraries
import torch
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from torch.nn.functional import softmax
import os
from pathlib import Path

c:\Users\Fawaz\Desktop\AI_Text_Detector\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Checking for gpu( Since one of us have a gpu)

In [2]:
# Set device (GPU if available, otherwise CPU)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Check CUDA availability
if torch.cuda.is_available():
    print("CUDA is available")
    print(f"Device count: {torch.cuda.device_count()}")
    print(f"Current device name: {torch.cuda.get_device_name(torch.cuda.current_device())}")
else:
    print("CUDA is not available, using CPU")

Using device: cpu
CUDA is not available, using CPU


# Loading the saved model from pt1

In [3]:
# Load the fine-tuned model and tokenizer
model_path = r"C:\Users\Fawaz\Desktop\AI_Text_Detector\human-ai-model"

# Load tokenizer
tokenizer = DistilBertTokenizerFast.from_pretrained(model_path)

# Load model
model = DistilBertForSequenceClassification.from_pretrained(model_path)
model.to(device)
model.eval()


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


# Prediction function with pre-processing

In [ ]:
# Define the prediction function
def predict(text: str, model, tokenizer):
    # Tokenizing the input text
    inputs = tokenizer(
        text, 
        return_tensors="pt", 
        truncation=True, 
        padding="max_length", 
        max_length=128
    ).to(device)
    
    # Making  prediction
    with torch.no_grad():
        outputs = model(**inputs)
        probs = softmax(outputs.logits, dim=1).squeeze()
        pred = torch.argmax(probs).item()
    
    # Prediction label and confidence
    label = "Human" if pred == 0 else "AI"
    confidence = probs[pred].item()
    
    return f"{label} (Confidence: {confidence:.3f})"

# Tesing the model(Replace your text in the input list to test it)

In [ ]:
# Test the model with some example texts
input = [
    """ With the growing chains of fast-food restaurants and street foods, the young generation today has become more addicted to eating outside and are dependent on these junk foods. This has become a new trend in modern culture. Also, the marketing of various food brands such as KFC, Pizza Hut, Domino’s and McDonald’s have attracted more youngsters to go and consume such food items. Although these items taste good but are not at all considered healthy for our bodies. With their higher level of proteins, sugar and saturated fats, they tend to increase the cholesterol level in our body. They do not contribute to providing nutritional values required for the immune system and lack essential vitamins and minerals. 



Due to the busy day-to-day life, we often prefer ordering food from outside or munching on some snacks to satisfy our hunger. As a result, this has increased our addiction to eating junk every day. We often enjoy eating food that satisfies our taste buds and is less time-consuming. Since these junk foods are easily found nowadays with just a click on our phone, we tend to eat more of them. But we should keep in mind that these create a harmful impact on our body later and give rise to several health problems.



As per reports, the increase in the number of obesity cases has resulted due to the regular consumption of junk food items. Some of the severe health issues include cardiovascular diseases, a higher risk of obesity, PCOD/PCOS in females and several other chronic diseases which have a greater impact on our body and mental health.



Some of the commonly eaten junk foods include pizzas, burgers, deep-fries, food items made of processed cheese and and high protein meat. Even some of the breakfast items and cereals tend to contain high amounts of sugar content.""",


"""
Junk food refers to foods that are high in calories, sugar, fat, and salt but low in essential nutrients like vitamins, minerals, and fiber. These foods are often quick and convenient, appealing to busy lifestyles, but they come with significant health risks when consumed regularly.

One of the primary concerns with junk food is its contribution to obesity. Many junk foods are calorically dense but offer little nutritional value, leading to overconsumption without providing the body with the nutrients it needs. This imbalance often results in weight gain and, over time, can contribute to serious health issues like heart disease, diabetes, and high blood pressure.

Additionally, junk food is often addictive due to its combination of fats, sugars, and salt, which trigger pleasure centers in the brain. This can lead to overeating and unhealthy eating habits. Processed foods like chips, sodas, and fast food meals also have a detrimental impact on the digestive system and may increase the risk of conditions like acid reflux and gastritis.

While junk food may offer a temporary sense of satisfaction, the long-term effects are harmful. In moderation, indulgence in these foods may not pose a significant threat, but consistent consumption can lead to poor health outcomes. As a result, it's crucial to prioritize a balanced diet rich in fruits, vegetables, whole grains, and lean proteins to ensure overall health and well-being.
"""

]

for i, text in enumerate(input, 1):
    prediction = predict(text, model, tokenizer)
    print(f"\n{i}. Text: {text}")
    print(f"   Prediction: {prediction}")


1. Text:  With the growing chains of fast-food restaurants and street foods, the young generation today has become more addicted to eating outside and are dependent on these junk foods. This has become a new trend in modern culture. Also, the marketing of various food brands such as KFC, Pizza Hut, Domino’s and McDonald’s have attracted more youngsters to go and consume such food items. Although these items taste good but are not at all considered healthy for our bodies. With their higher level of proteins, sugar and saturated fats, they tend to increase the cholesterol level in our body. They do not contribute to providing nutritional values required for the immune system and lack essential vitamins and minerals. 



Due to the busy day-to-day life, we often prefer ordering food from outside or munching on some snacks to satisfy our hunger. As a result, this has increased our addiction to eating junk every day. We often enjoy eating food that satisfies our taste buds and is less time